In [1]:
import numpy       as np
import pandas      as pd
import scipy.stats as stats

In [3]:
alpha = 0.05

##------------------------------------------------------------------------------
## Chuẩn bị dữ liệu
##------------------------------------------------------------------------------
## Kích thước của các mẫu có thể KHÁC NHAU
LDS3folder = 'E:\Data Science\Mathematics and Statistics for Data Science\Excercise'
folder = LDS3folder + '\Data\B7/'

fname= 'Excavation Depth and Archaeology.txt'
d          = pd.read_csv(folder + fname, sep = '\t')

## Xác định k nhóm là k cột trong tập dữ liệu
# groupsA = list(d.columns.values) # array
k      = len(d.columns)
groups = list(d.columns) # (k column headers)

## Tạo k mẫu (loại bỏ giá trị NaN trong các mẫu)
samples = []
for j in range(k):
    sample = [x for x in d[groups[j]] if pd.notnull(x)]
    samples.append(sample)


In [6]:
##------------------------------------------------------------------------------
print('----------- Cách 1: Tính toán "truyền thống" theo các công thức ---------')
##------------------------------------------------------------------------------    
## Số phần tử của mỗi nhóm
sizes = np.zeros(k)
for j in range(k): 
    sizes[j] = np.size(samples[j])

## Giá trị trung bình của mỗi mẫu
means = np.zeros(k)
for j in range(k):
    means[j] = np.mean(samples[j])

## Giá trị trung bình của tất cả các mẫu
meanT = np.mean(means)

## Các đại lượng BETWEEN groups: SSB, dfB
SSB = 0
for j in range(k):
    SSB += sizes[j] * np.power((means[j] - meanT), 2)
dfB = (k - 1)
print('SSB           : %.4f' %SSB)
print('dfB           : %d' %dfB)
   
## Các đại lượng WITHIN groups: SSW, dfW
SSW = 0
for j in range(k):
    SSWj = 0
    for i in range(int(sizes[j])):
        SSWj += np.power(samples[j][i] - means[j], 2)
    SSW += SSWj
dfW = int(np.sum(sizes) - k)
print('SSW           : %.4f' %SSW)
print('dfW           : %d' %dfW)

## Trị thống kê: F statistics
F = (SSB / dfB) / (SSW / dfW)
print('F statistic   : %.4f' %F)

## Giá trị tới hạn
##------------------------------------------------------------------------------
## Hàm scipy.stats.f.ppf(q, dfn, dfd) xác định giá trị tới hạn
##    q  : confidence level     (1 - alpha)
##    dfn: tử số (numerator)    dfB (BETWEEN groups)
##    dfd: mẫu số (denominator) dfW (WITHIN groups)
##
## Hàm scipy.stats.f.cdf(crit, dfn, dfd) xác định confidence level (1 - alpha)
##------------------------------------------------------------------------------
critical_value = stats.f.ppf(q = 1 - alpha, dfn = dfB, dfd = dfW)
print('Critical value: %.4f' %critical_value)
conf_level     = stats.f.cdf(critical_value, dfn = dfB, dfd = dfW)

if (F < critical_value):
    print('(F <  critical value) ==> ACCEPT the H0 that the means are equal.')
else:
    print('(F >= critical value) ==> REJECT the H0 that the means are equal.')

----------- Cách 1: Tính toán "truyền thống" theo các công thức ---------
SSB           : 12486.4848
dfB           : 3
SSW           : 11464.5722
dfW           : 42
F statistic   : 15.2479
Critical value: 2.8270
(F >= critical value) ==> REJECT the H0 that the means are equal.


In [7]:
##------------------------------------------------------------------------------
print('\n--------------------- Cách 2: Sử dụng hàm của Python ------------------')
##------------------------------------------------------------------------------    
fvalue, pvalue = stats.f_oneway(samples[0], samples[1], samples[2], samples[3])

print('p-value       : %.4f' %pvalue)
print('F statistic   : %.4f' %fvalue)


--------------------- Cách 2: Sử dụng hàm của Python ------------------
p-value       : 0.0000
F statistic   : 15.1391
